# Week 5: GUI Integration with Gradio
## Complete Customer Review Analyzer Application

**Focus:** Build user-friendly interface combining sentiment analysis + RAG

---

## Application Overview

**Features:**
1. **Sentiment Analysis Tab:** Analyze customer review sentiment
2. **Product Q&A Tab:** Ask questions about products using RAG
3. **Combined Analysis:** Both features in one interface

**User Flow:**
```
User → Gradio Interface → Sentiment Model / RAG System → Results Display
```

---

## Step 1: Install All Required Libraries

In [ ]:
# Install all required packages
!pip install -q transformers datasets torch sentence-transformers chromadb gradio

print("✅ All libraries installed")

## Step 2: Import Libraries and Setup

In [ ]:
# Import all necessary libraries
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

## Step 3: Load/Train Sentiment Model

We'll use a simplified training process for quick demo purposes.

In [ ]:
# Load and prepare sentiment model
print("Loading sentiment analysis model...")

# Option 1: If you have a saved model from Week 2
# sentiment_tokenizer = AutoTokenizer.from_pretrained("./sentiment_model")
# sentiment_model = AutoModelForSequenceClassification.from_pretrained("./sentiment_model")

# Option 2: Quick training for demo (recommended for Colab)
sentiment_model_name = "distilbert-base-uncased"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(
    sentiment_model_name,
    num_labels=2
)

# Quick fine-tuning (500 samples, 1 epoch)
print("Quick training on review data...")
train_data = load_dataset("amazon_polarity", split="train[:500]")

def preprocess_and_tokenize(examples):
    texts = [f"{title}. {content[:200]}" for title, content in zip(examples['title'], examples['content'])]
    return sentiment_tokenizer(texts, padding='max_length', truncation=True, max_length=128)

tokenized_data = train_data.map(preprocess_and_tokenize, batched=True)
tokenized_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    logging_steps=50,
    save_strategy="no"
)

trainer = Trainer(
    model=sentiment_model,
    args=training_args,
    train_dataset=tokenized_data
)

trainer.train()

print("✅ Sentiment model ready")

## Step 4: Setup RAG System

In [ ]:
# Setup RAG components
print("Setting up RAG system...")

# Load embedding model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Create company documents
documents = [
    {
        "id": "doc1",
        "title": "Product Specifications",
        "content": """
        Premium Wireless Headphones Specifications:
        Battery Life: 30 hours on single charge
        Active Noise Cancellation (ANC): Yes
        Bluetooth: Version 5.0, 10-meter range
        Compatibility: iOS and Android
        Design: Foldable with carrying case
        Colors: Black, Silver, Rose Gold
        Price: $149.99
        Weight: 250 grams
        Charging: USB-C, 2 hours full charge
        """
    },
    {
        "id": "doc2",
        "title": "Setup and Usage",
        "content": """
        How to pair headphones:
        1. Enable Bluetooth on your device
        2. Press and hold power button for 3 seconds
        3. LED will flash blue
        4. Select 'Premium Headphones' in Bluetooth menu
        
        Using while charging: Yes, use wired mode with 3.5mm cable
        Voice assistants: Compatible with Siri, Google Assistant, Alexa
        """
    },
    {
        "id": "doc3",
        "title": "Warranty and Returns",
        "content": """
        Return Policy:
        - 30-day money-back guarantee
        - Must include original packaging
        - Free return shipping on defects
        - Refund in 5-7 business days
        
        Warranty:
        - 2-year manufacturer warranty
        - Covers defects and hardware failures
        - Does not cover physical/water damage
        - Contact: support@company.com
        """
    },
    {
        "id": "doc4",
        "title": "Troubleshooting",
        "content": """
        Common Problems:
        
        Won't turn on: Charge 30 minutes, or hard reset (hold power 10 seconds)
        Poor sound: Check ear cushion fit, clean speaker mesh, disable ANC
        Connection drops: Stay within 10m range, remove obstacles, re-pair
        Mic not working: Check permissions, unmute (volume down x3)
        Battery drains fast: Disable ANC when not needed, fully power off
        """
    }
]

# Initialize ChromaDB
chroma_client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    allow_reset=True
))

# Create collection
try:
    chroma_client.delete_collection("product_docs")
except:
    pass

collection = chroma_client.create_collection(name="product_docs")

# Add documents to vector database
for doc in documents:
    embedding = embedding_model.encode(doc['content']).tolist()
    collection.add(
        ids=[doc['id']],
        embeddings=[embedding],
        documents=[doc['content']],
        metadatas=[{"title": doc['title']}]
    )

print("✅ RAG system ready")
print(f"   Documents loaded: {len(documents)}")

## Step 5: Create Application Functions

In [ ]:
# Sentiment analysis function for GUI
# AI Assistance: Claude helped create user-friendly output formatting

def analyze_sentiment(review_text):
    """
    Analyzes sentiment of customer review.
    Returns formatted results for GUI display.
    """
    if not review_text.strip():
        return "⚠️ Please enter a review to analyze.", None
    
    # Tokenize input
    inputs = sentiment_tokenizer(
        review_text,
        return_tensors="pt",
        truncation=True,
        max_length=128
    )
    
    # Get prediction
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Extract results
    predicted_class = predictions.argmax().item()
    confidence = predictions[0][predicted_class].item()
    
    # Format output
    sentiment_label = "😊 Positive" if predicted_class == 1 else "😞 Negative"
    confidence_pct = f"{confidence * 100:.1f}%"
    
    result_text = f"""
### Analysis Results

**Sentiment:** {sentiment_label}
**Confidence:** {confidence_pct}

**Review Analyzed:**
{review_text}
"""
    
    # Create confidence breakdown
    positive_conf = predictions[0][1].item() * 100
    negative_conf = predictions[0][0].item() * 100
    
    confidence_data = {
        "Positive": positive_conf,
        "Negative": negative_conf
    }
    
    return result_text, confidence_data


# RAG query function for GUI
def answer_question(question):
    """
    Answers product questions using RAG.
    Returns formatted results for GUI display.
    """
    if not question.strip():
        return "⚠️ Please enter a question."
    
    # Embed question
    question_embedding = embedding_model.encode(question).tolist()
    
    # Search vector database
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=2
    )
    
    # Extract results
    retrieved_docs = results['documents'][0]
    retrieved_metadata = results['metadatas'][0]
    
    # Generate answer
    context = "\n".join(retrieved_docs)
    answer = extract_answer_simple(question, context)
    
    # Format output
    sources = [meta['title'] for meta in retrieved_metadata]
    
    result_text = f"""
### 💡 Answer

{answer}

### 📚 Sources
"""
    for i, source in enumerate(sources, 1):
        result_text += f"\n{i}. {source}"
    
    return result_text


def extract_answer_simple(question, context):
    """
    Simple answer extraction from retrieved context.
    """
    # Split into sentences
    sentences = [s.strip() + '.' for s in context.split('.') if s.strip()]
    
    # Find relevant sentences
    question_words = set(question.lower().split())
    question_words -= {'what', 'how', 'when', 'where', 'why', 'is', 'are', 'the', 'a', 'an', 'do', 'does'}
    
    # Score and rank sentences
    scored = []
    for sentence in sentences:
        words = set(sentence.lower().split())
        score = len(question_words & words)
        if score > 0:
            scored.append((score, sentence))
    
    scored.sort(reverse=True)
    
    # Take top 3 sentences
    if scored:
        answer_sentences = [s[1] for s in scored[:3]]
        return ' '.join(answer_sentences)
    else:
        return "I found relevant information but couldn't extract a specific answer. Please try rephrasing your question."


print("✅ Application functions created")

## Step 6: Build Gradio Interface

In [ ]:
# Create Gradio interface with tabs
# AI Assistance: Claude designed the multi-tab interface structure

# Custom CSS for better styling
custom_css = """
.gradio-container {
    font-family: 'Arial', sans-serif;
}
.output-markdown {
    padding: 20px;
    background-color: #f8f9fa;
    border-radius: 8px;
}
"""

# Create interface with tabs
with gr.Blocks(css=custom_css, title="Customer Review Analyzer") as demo:
    
    # Header
    gr.Markdown(
        """
        # 🎧 Customer Review Analyzer
        ### AI-Powered Sentiment Analysis + Product Knowledge Assistant
        
        Analyze customer reviews and get instant answers about your products!
        """
    )
    
    # Tab 1: Sentiment Analysis
    with gr.Tab("📊 Sentiment Analysis"):
        gr.Markdown("""
        ### Analyze Customer Review Sentiment
        Paste a customer review to determine if it's positive or negative.
        """)
        
        with gr.Row():
            with gr.Column():
                sentiment_input = gr.Textbox(
                    label="Customer Review",
                    placeholder="Enter customer review here...",
                    lines=5
                )
                sentiment_button = gr.Button("Analyze Sentiment", variant="primary")
                
                # Example reviews
                gr.Examples(
                    examples=[
                        ["These headphones are amazing! Great sound quality and battery life."],
                        ["Terrible product. Broke after one week. Very disappointed."],
                        ["Pretty good for the price. Sound is decent but not exceptional."],
                    ],
                    inputs=sentiment_input,
                    label="Try these examples:"
                )
            
            with gr.Column():
                sentiment_output = gr.Markdown(label="Results")
                sentiment_chart = gr.Label(label="Confidence Breakdown")
        
        sentiment_button.click(
            fn=analyze_sentiment,
            inputs=sentiment_input,
            outputs=[sentiment_output, sentiment_chart]
        )
    
    # Tab 2: Product Q&A (RAG)
    with gr.Tab("❓ Product Q&A"):
        gr.Markdown("""
        ### Ask Questions About Products
        Get instant answers from product documentation using AI retrieval.
        """)
        
        with gr.Row():
            with gr.Column():
                question_input = gr.Textbox(
                    label="Your Question",
                    placeholder="What would you like to know?",
                    lines=3
                )
                question_button = gr.Button("Get Answer", variant="primary")
                
                # Example questions
                gr.Examples(
                    examples=[
                        ["What is the battery life?"],
                        ["How do I pair the headphones?"],
                        ["What is the return policy?"],
                        ["What colors are available?"],
                        ["My headphones won't turn on, what should I do?"],
                    ],
                    inputs=question_input,
                    label="Try these questions:"
                )
            
            with gr.Column():
                answer_output = gr.Markdown(label="Answer")
        
        question_button.click(
            fn=answer_question,
            inputs=question_input,
            outputs=answer_output
        )
    
    # Tab 3: About
    with gr.Tab("ℹ️ About"):
        gr.Markdown("""
        ## About This Application
        
        ### 🎯 Purpose
        This AI system helps businesses:
        - **Analyze** customer review sentiment automatically
        - **Answer** product questions using company knowledge
        - **Save time** on customer service tasks
        
        ### 🛠️ Technical Components
        
        **1. Sentiment Analysis Model**
        - Model: DistilBERT (distilbert-base-uncased)
        - Task: Binary sentiment classification (Positive/Negative)
        - Training: Fine-tuned on Amazon product reviews
        
        **2. RAG (Retrieval-Augmented Generation) System**
        - Embeddings: MiniLM-L6-v2 (Sentence Transformers)
        - Vector Database: ChromaDB
        - Documents: Product specs, FAQs, policies, troubleshooting
        
        **3. User Interface**
        - Framework: Gradio
        - Features: Multi-tab interface with examples
        
        ### 📚 Data Sources
        - Amazon Product Reviews (Hugging Face)
        - Custom product documentation
        
        ### 🤖 AI Assistance
        This project was developed with assistance from Claude (Anthropic):
        - Code structure and implementation
        - Documentation and comments
        - Interface design
        
        ### 📝 Course Information
        - **Course:** INFO 452 - AI Applications
        - **Instructor:** Dr. Ben Marlin
        - **Semester:** Fall 2025
        - **Project:** Final Project (Weeks 1-5)
        
        ### ⚖️ Citations
        - DistilBERT: Sanh et al. (2019)
        - Amazon Reviews: Zhang et al. (2015)
        - Sentence Transformers: Reimers & Gurevych (2019)
        - ChromaDB: Chroma (2023)
        - Gradio: Abid et al. (2019)
        """)
    
    # Footer
    gr.Markdown("""
    ---
    **Note:** This is a demonstration system for educational purposes. 
    Sentiment predictions and answers may not always be accurate.
    """)

print("✅ Gradio interface created")

## Step 7: Launch Application

In [ ]:
# Launch the Gradio app
# This will create a public URL that you can share

print("Launching application...")
print("\n" + "="*70)
print("🚀 Your AI application is ready!")
print("="*70 + "\n")

# Launch with public link (set share=False for local only)
demo.launch(
    share=True,  # Creates public URL
    debug=True   # Shows detailed error messages
)

# The application will be available at the URL shown above

## Step 8: Testing Checklist

Before submitting, test these scenarios:

### Sentiment Analysis
- ✅ Test positive review
- ✅ Test negative review
- ✅ Test neutral/mixed review
- ✅ Check confidence scores

### Product Q&A
- ✅ Ask about specifications
- ✅ Ask about troubleshooting
- ✅ Ask about warranty/returns
- ✅ Verify source citations

### Interface
- ✅ All tabs work
- ✅ Examples are clickable
- ✅ Output formatting is clear
- ✅ Error handling works

## Week 5 Summary

**Completed:**
- ✅ Integrated sentiment model with GUI
- ✅ Integrated RAG system with GUI
- ✅ Built multi-tab Gradio interface
- ✅ Added example inputs for both features
- ✅ Created comprehensive About page
- ✅ Tested full application pipeline
- ✅ Generated shareable public link

**Application Features:**
1. **Sentiment Analysis Tab:**
   - Analyzes review text
   - Shows positive/negative prediction
   - Displays confidence scores
   - Provides confidence breakdown chart

2. **Product Q&A Tab:**
   - Answers product questions
   - Retrieves from vector database
   - Cites source documents
   - Fast response time

3. **About Tab:**
   - Technical documentation
   - Component descriptions
   - Citations and credits

**Deployment Options:**
1. **Current:** Gradio share link (temporary)
2. **Permanent:** Deploy to Hugging Face Spaces (instructions in README)

**Next Steps:**
- Create README.md file
- Write reflection document
- Prepare presentation
- Deploy to Hugging Face Spaces

---

**AI Assistance Documentation:**
- Claude (Anthropic) provided:
  - Complete integration code
  - Gradio interface design
  - User experience optimization
  - Error handling implementation
  - Documentation and styling

**Final Citations:**
- All models and libraries cited in About tab
- Amazon Reviews dataset from Hugging Face
- Open-source tools used throughout